Written by Zain Kamal [zain.eris.kamal@rutgers.edu](mailto:zain.eris.kamal@rutgers.edu) on 03/12/2024.

https://github.com/Humboldt-Penguin/redplanet

Rewriting Crust module following `GRS.py`, since I found out that xarray is blazingly fast for accessing large swaths of data, and enables an array-based workflow that will speed up calculations by multiple orders of magnitude (from 40+ seconds to less than a second!!!).

---
# [0] Setup

In [1]:
from redplanet import utils

from pathlib import Path
import json

import pooch
import numpy as np
import pandas as pd 
import xarray as xr
import pyshtools as pysh

In [2]:
import matplotlib.pyplot as plt

---
---
# [1] Download

---
## [1.1] Dichotomy Coords

In [3]:
# dat_dichotomy_coords = None

# def _load_dichotomy():

#     global dat_dichotomy_coords

#     '''download / cache'''
#     with utils.disable_pooch_logger():
#         fpath_dichotomy_coords = pooch.retrieve(
#             fname      = 'dichotomy_coordinates-JAH-0-360.txt',
#             url        = r'https://drive.google.com/file/d/17exPNRMKXGwa3daTEBN02llfdya6OZJY/view?usp=sharing',
#             known_hash = 'sha256:42f2b9f32c9e9100ef4a9977171a54654c3bf25602555945405a93ca45ac6bb2',
#             path       = pooch.os_cache('redplanet') / 'Crust' / 'dichotomy',
#             downloader = utils.download_gdrive_file,
#         )
#     fpath_dichotomy_coords = Path(fpath_dichotomy_coords)



#     '''load into Nx2 numpy array of dichotomy coordinates, structured (lon, lat).'''
#     dat_dichotomy_coords = np.loadtxt(fpath_dichotomy_coords)

#     ## fix the lons
#     dat_dichotomy_coords[:,0] = utils.clon2lon(dat_dichotomy_coords[:,0])
#     dat_dichotomy_coords = dat_dichotomy_coords[np.argsort(dat_dichotomy_coords[:,0])]

#     ## add wraparound coordinates for safety /convenience
#     dat_dichotomy_coords = np.vstack((
#         dat_dichotomy_coords, 
#         [dat_dichotomy_coords[0,0]+360, dat_dichotomy_coords[0,1]], 
#         [dat_dichotomy_coords[1,0]+360, dat_dichotomy_coords[1,1]], 
#     ))

#     dat_dichotomy_coords


In [4]:
# _load_dichotomy()

# plt.scatter(x=dat_dichotomy_coords[:,0], y=dat_dichotomy_coords[:,1])

---
## [1.2] Topography

In [5]:
'''
Manually loading topography is slightly inconvenient:
    - Spherical harmonic coefficient file 'MarsTopo2600.shape.gz' is 73MB compressed, or 200MB decompressed. 
    - Loading this and expanding to 0.1 degree grid spacing takes 3-7 seconds, which can be annoying for users if forced to wait everytime they import `redplanet.Crust`. 
So instead, I pre-compute the `pysh.ShGrid` object and save it to a numpy binary for speed -- corresponding code is below. This topography grid is loaded by default, and if the user wants a finer resolution, we download/load/expand manually.
'''

from redplanet import utils
import pyshtools as pysh
import pooch
from pathlib import Path

grid_spacing = 0.1

lmax = round(90. / grid_spacing - 1)
grid_spacing = 180. / (2 * lmax + 2)

with utils.disable_pooch_logger():
    fpath_MarsTopo2600 = pooch.retrieve(
        fname      = 'MarsTopo2600.shape.gz',
        url        = r'https://drive.google.com/file/d/1so3sGXNzdYkTdpzjvOxwYBsvr1Y1lwXt/view?usp=sharing',
        known_hash = 'sha256:8882a9ee7ee405d971b752028409f69bd934ba5411f1c64eaacd149e3b8642af',
        path       = pooch.os_cache('redplanet') / 'Crust',
        downloader = utils.download_gdrive_file,
        processor  = pooch.Decompress(),
    )

topo_shcoeffs = pysh.SHCoeffs.from_file(fpath_MarsTopo2600, lmax=lmax, name='MarsTopo2600', units='m')
topo_shgrid = topo_shcoeffs.expand(grid='DH2', extend=True) * 1e-3 # convert m -> km

fpath = Path.cwd() / 'pysh-ShGrid_MarsTopo2600_0.1deg_km.npy'
topo_shgrid.to_file(fpath, binary=True)
print(f'sha256:{pooch.file_hash(fpath)}')

KeyboardInterrupt: 

In [ ]:
dat_crust_xrds = None



def load_topo(grid_spacing=0.1):

    global dat_crust_xrds

    lmax = round(90. / grid_spacing - 1)
    grid_spacing = 180. / (2 * lmax + 2)


    if grid_spacing == 0.1:
        '''use pre-computed grid for speed -- see bottom of file for further discussion on this.'''
        with utils.disable_pooch_logger():
            fpath_topo_grid = pooch.retrieve(
                fname      = 'pysh-ShGrid_MarsTopo2600_0.1deg_km.npy',
                url        = r'https://drive.google.com/file/d/10m0S4eunb05jkOf4rwnxiWAToLVpt1hq/view?usp=sharing',
                known_hash = 'sha256:1a7e7fdfc23b8b8d68c115469888fcf304957ec681ebe12295d7e8cef31feb61',
                path       = pooch.os_cache('redplanet') / 'Crust' / 'topo',
                downloader = utils.download_gdrive_file,
            )

        # topo_grid = Path(topo_grid)  # causes error when trying to readfile with pysh.SHCoeffs.from_file, it expects a string
        topo_shgrid = pysh.SHGrid.from_file(fpath_topo_grid, binary=True)


    else:
        '''if user is manually requesting a finer grid, compute manually -- the overhead is downloading additional ~300MB to cache and 3-10 seconds processing, so recommend avoiding this.'''
        with utils.disable_pooch_logger():
            fpath_MarsTopo2600 = pooch.retrieve(
                fname      = 'MarsTopo2600.shape.gz',
                url        = r'https://drive.google.com/file/d/1so3sGXNzdYkTdpzjvOxwYBsvr1Y1lwXt/view?usp=sharing',
                known_hash = 'sha256:8882a9ee7ee405d971b752028409f69bd934ba5411f1c64eaacd149e3b8642af',
                path       = pooch.os_cache('redplanet') / 'Crust' / 'topo',
                downloader = utils.download_gdrive_file,
                processor  = pooch.Decompress(),
            )

        # fpath_MarsTopo2600 = Path(fpath_MarsTopo2600)  # causes error when trying to readfile with pysh.SHCoeffs.from_file, it expects a string
        topo_shcoeffs = pysh.SHCoeffs.from_file(fpath_MarsTopo2600, lmax=lmax, name='MarsTopo2600', units='m')
        topo_shgrid = topo_shcoeffs.expand(grid='DH2', extend=True) * 1e-3 # convert m -> km


    topo_xrda = fix_xarray_coords(topo_shgrid.to_xarray())
    dat_crust_xrds = xr.Dataset({'topo': topo_xrda})
    dat_crust_xrds.attrs = {
        'units': 'km',
        'grid_spacing': grid_spacing, 
        'lmax': lmax,
        'topo_model': 'MarsTopo2600',
    }




def fix_xarray_coords(dataarray):
    dataarray = dataarray.assign_coords(lon=xr.apply_ufunc(utils.clon2lon, dataarray.lon))
    dataarray = dataarray.sortby('lon', ascending=True)
    dataarray = dataarray.sortby('lat', ascending=True)
    return dataarray


In [ ]:
load_topo()

In [ ]:
lons = np.arange(0,360,1)
lats = np.arange(-90,90,1)

lons = np.round(lons,10)
lats = np.round(lats,10)

dat = dat_crust_xrds.topo.sel(lon=lons, lat=lats, method='nearest')
dat.plot(figsize=(12,6))

In [ ]:
lons = np.arange(20,27,grid_spacing)
lats = np.arange(7,14,grid_spacing)

lons = np.round(lons,10)
lats = np.round(lats,10)

dat = dat_crust_xrds.topo.sel(lon=lons, lat=lats, method='nearest')
dat.plot()

---
## [1.3] Moho

In [17]:
# def load_model(RIM, insight_thickness, rho_north, rho_south, suppress_model_error=False):

#     global dat_crust_xrds

#     model_name = f'{RIM}-{insight_thickness}-{rho_south}-{rho_north}'



#     '''load a registry of moho models, which provides a google drive download link and a sha256 hash for a given model name'''

#     with utils.disable_pooch_logger():
#         fpath_moho_shcoeffs_registry = pooch.retrieve(
#             fname      = 'Crust_mohoSHcoeffs_rawdata_registry.json',
#             url        = r'https://drive.google.com/file/d/17JJuTFKkHh651-rt2J2eFKnxiki0w4ue/view?usp=sharing',
#             known_hash = 'sha256:1800ee2883dc6bcc82bd34eb2eebced5b59fbe6c593cbc4e9122271fd01c1491',
#             path       = pooch.os_cache('redplanet') / 'Crust' / 'moho', 
#             downloader = utils.download_gdrive_file,
#         )

#     with open(fpath_moho_shcoeffs_registry, 'r') as file:
#         moho_shcoeffs_registry = json.load(file)



#     '''download SH coefficients for the chosen model'''

#     try:
#         _ = moho_shcoeffs_registry[model_name]
#     except KeyError:
#         if suppress_model_error:
#             return False
#         else:
#             raise ValueError(f'No Moho model with the inputs {model_name} exists.')

#     with utils.disable_pooch_logger():
#         fpath_moho_shcoeffs = pooch.retrieve(
#             fname      = f'{model_name}.txt',
#             url        = moho_shcoeffs_registry[model_name]['link'], 
#             known_hash = moho_shcoeffs_registry[model_name]['hash'],
#             path       = pooch.os_cache('redplanet') / 'Crust' / 'moho' / 'SH_coeffs', 
#             downloader = utils.download_gdrive_file, 
#         )



#     '''load+save model'''
#     lmax = dat_crust_xrds.lmax
#     moho_shcoeffs = pysh.SHCoeffs.from_file(fpath_moho_shcoeffs)
#     moho_shgrid = moho_shcoeffs.expand(lmax=lmax, grid='DH2', extend=True) * 1e-3 # convert m -> km

#     moho_xrda = fix_xarray_coords(moho_shgrid.to_xarray())
#     dat_crust_xrds['moho'] = moho_xrda
#     moho_attrs = {
#         'moho_model_name': model_name,
#         'moho_model_RIM': RIM,
#         'moho_model_insight_thickness': insight_thickness,
#         'moho_model_rho_north': rho_north,
#         'moho_model_rho_south': rho_south,
#     }
#     dat_crust_xrds.attrs.update(moho_attrs)


In [18]:
# RIM               = 'Khan2022'
# insight_thickness = 39
# rho_north         = 2900
# rho_south         = 2900

# load_model(RIM, insight_thickness, rho_north, rho_south)

In [27]:
# dat_crust_xrds

<xarray.Dataset> Size: 104MB
Dimensions:  (lat: 1801, lon: 3601)
Coordinates:
  * lat      (lat) float64 14kB -90.0 -89.9 -89.8 -89.7 ... 89.7 89.8 89.9 90.0
  * lon      (lon) float64 29kB -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
Data variables:
    topo     (lat, lon) float64 52MB 3.382e+03 3.382e+03 ... 3.376e+03 3.376e+03
    moho     (lat, lon) float64 52MB 3.309e+03 3.309e+03 ... 3.343e+03 3.343e+03
Attributes:
    units:                         km
    grid_spacing:                  0.1
    lmax:                          899
    topo_model:                    MarsTopo2600
    moho_model:                    Khan2022-39-2900-2900
    moho_model_name:               Khan2022-39-2900-2900
    moho_model_RIM:                Khan2022
    moho_model_insight_thickness:  39
    moho_model_rho_north:          2900
    moho_model_rho_south:          2900

---
---
# [2] Initializer

In [28]:
## lazy initialization.
_has_been_initialized = False

## holds Crust data in xarray dataset or dictionary format.
dat_crust_xrds = None
dat_crust_dict = None

dat_dichotomy_coords = None


def get_rawdata(how):
    """
    `format` options: ['xarray', 'dict', 'dichotomy']

    Note: when viewing/exploring dictionaries, it may help to call:
        ```
        from redplanet import utils
        utils.print_dict(dat_something_dict)     # insert any dictionary here
        ```
    """
    match how:
        case 'xarray':
            return dat_crust_xrds
        case 'dict':
            return dat_crust_dict
        case 'dichotomy':
            return dat_dichotomy_coords
        case _:
            raise ValueError('Options are ["xarray", "dict", "dichotomy"].')








def _initialize():

    '''lazy initialization uwu'''    
    global _has_been_initialized
    if _has_been_initialized:
        return

    '''load topo+dichotomy only -- user should consciously think about which moho to choose, rather than getting an arbitrary model for free'''
    load_topo()
    _load_dichotomy()

    '''lazy loadinggg'''
    _has_been_initialized = True

    return







def _load_dichotomy():

    global dat_dichotomy_coords

    '''download / cache'''
    with utils.disable_pooch_logger():
        fpath_dichotomy_coords = pooch.retrieve(
            fname      = 'dichotomy_coordinates-JAH-0-360.txt',
            url        = r'https://drive.google.com/file/d/17exPNRMKXGwa3daTEBN02llfdya6OZJY/view?usp=sharing',
            known_hash = 'sha256:42f2b9f32c9e9100ef4a9977171a54654c3bf25602555945405a93ca45ac6bb2',
            path       = pooch.os_cache('redplanet') / 'Crust' / 'dichotomy',
            downloader = utils.download_gdrive_file,
        )
    fpath_dichotomy_coords = Path(fpath_dichotomy_coords)



    '''load into Nx2 numpy array of dichotomy coordinates, structured (lon, lat).'''
    dat_dichotomy_coords = np.loadtxt(fpath_dichotomy_coords)

    ## fix the lons (convert from 0->360 to -180->180 and sort)
    dat_dichotomy_coords[:,0] = utils.clon2lon(dat_dichotomy_coords[:,0])
    dat_dichotomy_coords = dat_dichotomy_coords[np.argsort(dat_dichotomy_coords[:,0])]

    ## add wraparound coordinates for safety / convenience
    dat_dichotomy_coords = np.vstack((
        dat_dichotomy_coords, 
        [dat_dichotomy_coords[0,0]+360, dat_dichotomy_coords[0,1]], 
        [dat_dichotomy_coords[1,0]+360, dat_dichotomy_coords[1,1]], 
    ))












def load_topo(grid_spacing=0.1):

    global dat_crust_xrds

    lmax = round(90. / grid_spacing - 1)
    grid_spacing = 180. / (2 * lmax + 2)


    if grid_spacing == 0.1:
        '''use pre-computed grid for speed -- see bottom of file for further discussion on this.'''
        with utils.disable_pooch_logger():
            fpath_topo_grid = pooch.retrieve(
                fname      = 'pysh-ShGrid_MarsTopo2600_0.1deg_km.npy',
                url        = r'https://drive.google.com/file/d/10m0S4eunb05jkOf4rwnxiWAToLVpt1hq/view?usp=sharing',
                known_hash = 'sha256:1a7e7fdfc23b8b8d68c115469888fcf304957ec681ebe12295d7e8cef31feb61',
                path       = pooch.os_cache('redplanet') / 'Crust' / 'topo',
                downloader = utils.download_gdrive_file,
            )

        # topo_grid = Path(topo_grid)  # causes error when trying to readfile with pysh.SHCoeffs.from_file, it expects a string
        topo_shgrid = pysh.SHGrid.from_file(fpath_topo_grid, binary=True)


    else:
        '''if user is manually requesting a finer grid, compute manually -- the overhead is downloading additional ~300MB to cache and 3-10 seconds processing, so recommend avoiding this.'''
        with utils.disable_pooch_logger():
            fpath_MarsTopo2600 = pooch.retrieve(
                fname      = 'MarsTopo2600.shape.gz',
                url        = r'https://drive.google.com/file/d/1so3sGXNzdYkTdpzjvOxwYBsvr1Y1lwXt/view?usp=sharing',
                known_hash = 'sha256:8882a9ee7ee405d971b752028409f69bd934ba5411f1c64eaacd149e3b8642af',
                path       = pooch.os_cache('redplanet') / 'Crust' / 'topo',
                downloader = utils.download_gdrive_file,
                processor  = pooch.Decompress(),
            )

        # fpath_MarsTopo2600 = Path(fpath_MarsTopo2600)  # causes error when trying to readfile with pysh.SHCoeffs.from_file, it expects a string
        topo_shcoeffs = pysh.SHCoeffs.from_file(fpath_MarsTopo2600, lmax=lmax, name='MarsTopo2600', units='m')
        topo_shgrid = topo_shcoeffs.expand(grid='DH2', extend=True) * 1e-3 # convert m -> km


    '''format into xarray dataset'''
    topo_xrda = _fix_xarray_coords(topo_shgrid.to_xarray())
    dat_crust_xrds = xr.Dataset({'topo': topo_xrda})
    dat_crust_xrds.attrs = {
        'units': 'km',
        'grid_spacing': grid_spacing, 
        'lmax': lmax,
        'topo_model': 'MarsTopo2600',
    }


    _update_dict_to_match_xrds()







def load_model(RIM, insight_thickness, rho_north, rho_south, suppress_model_error=False) -> bool:

    '''lazy initialization'''
    _initialize()


    global dat_crust_xrds
    
    model_name = f'{RIM}-{insight_thickness}-{rho_south}-{rho_north}'



    '''load a pre-computed registry of moho models, which provides a google drive download link and a sha256 hash for a given model name'''
    with utils.disable_pooch_logger():
        fpath_moho_shcoeffs_registry = pooch.retrieve(
            fname      = 'Crust_mohoSHcoeffs_rawdata_registry.json',
            url        = r'https://drive.google.com/file/d/17JJuTFKkHh651-rt2J2eFKnxiki0w4ue/view?usp=sharing',
            known_hash = 'sha256:1800ee2883dc6bcc82bd34eb2eebced5b59fbe6c593cbc4e9122271fd01c1491',
            path       = pooch.os_cache('redplanet') / 'Crust' / 'moho', 
            downloader = utils.download_gdrive_file,
        )

    with open(fpath_moho_shcoeffs_registry, 'r') as file:
        moho_shcoeffs_registry = json.load(file)



    '''download SH coefficients for the chosen model'''
    try:
        _ = moho_shcoeffs_registry[model_name]
    except KeyError:
        if suppress_model_error:
            return False
        else:
            raise ValueError(f'No Moho model with the inputs {model_name} exists.')

    with utils.disable_pooch_logger():
        fpath_moho_shcoeffs = pooch.retrieve(
            fname      = f'{model_name}.txt',
            url        = moho_shcoeffs_registry[model_name]['link'], 
            known_hash = moho_shcoeffs_registry[model_name]['hash'],
            path       = pooch.os_cache('redplanet') / 'Crust' / 'moho' / 'SH_coeffs', 
            downloader = utils.download_gdrive_file, 
        )



    '''load+save model'''
    lmax = dat_crust_xrds.lmax
    moho_shcoeffs = pysh.SHCoeffs.from_file(fpath_moho_shcoeffs)
    moho_shgrid = moho_shcoeffs.expand(lmax=lmax, grid='DH2', extend=True) * 1e-3 # convert m -> km

    moho_xrda = _fix_xarray_coords(moho_shgrid.to_xarray())
    dat_crust_xrds['moho'] = moho_xrda
    more_moho_attrs = {
        'moho_model_name': model_name,
        'moho_model_RIM': RIM,
        'moho_model_insight_thickness': insight_thickness,
        'moho_model_rho_north': rho_north,
        'moho_model_rho_south': rho_south,
    }
    dat_crust_xrds.attrs.update(more_moho_attrs)


    _update_dict_to_match_xrds()

    return True









def _fix_xarray_coords(dataarray):
    dataarray = dataarray.assign_coords(lon=xr.apply_ufunc(utils.clon2lon, dataarray.lon))
    dataarray = dataarray.sortby('lon', ascending=True)
    dataarray = dataarray.sortby('lat', ascending=True)
    return dataarray







def _update_dict_to_match_xrds():
    global dat_crust_dict
    dat_crust_dict = {}
    for var in list(dat_crust_xrds.variables):
        dat_crust_dict[var] = dat_crust_xrds[var].values
    dat_crust_dict['attrs'] = dat_crust_xrds.attrs

---
---
# [3] Getter Methods

---
---
# [4] Hillshade yaya :3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

def calculate_hillshade(array, azimuth, angle_altitude):
    x, y = np.gradient(array)
    slope = np.pi / 2. - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    azimuth_rad = azimuth * np.pi / 180.
    altitude_rad = angle_altitude * np.pi / 180.

    shaded = np.sin(altitude_rad) * np.sin(slope) \
             + np.cos(altitude_rad) * np.cos(slope) \
             * np.cos(azimuth_rad - aspect)

    return shaded

# Example topography data (replace this with your actual data)
# topography = np.random.rand(100, 100) * 2  # 2 km maximum elevation
topography = dat[::-1]

# Smooth the data for more realistic shading
# topography = gaussian_filter(topography, sigma=1)

# Calculate hillshade (azimuth and altitude are customizable)
hillshade = calculate_hillshade(topography, azimuth=0, angle_altitude=30)
# hillshade = topography

# Plotting
plt.figure(figsize=(7,7))
plt.imshow(hillshade, cmap='gray', alpha=1)
# plt.imshow(dat_grs, cmap='viridis', alpha=0.5)
# plt.title('Hillshade plot')
plt.axis('off')
plt.show()